# Simple interactive visualizer for an Hénon map
### We want to observe the local evolution of a group of particles with the same initial action $I_0$

## Setup scripts if we are under SWAN

In [ ]:
# Working in the right path
%cd /eos/project/d/da-and-diffusion-studies/DA_Studies/Simulations/Models/da_sixtrack

In [ ]:
# Install the libraries
import sys
!{sys.executable} -m pip install --user --upgrade --force-reinstall tqdm pynverse sixtrackwrap crank-nicolson-numba henon-map
!export PYTHONPATH=$CERNBOX_HOME/.local/lib/python3.7/site-packages:$PYTHONPATH

In [ ]:
# For this "presentation" only!
import warnings
warnings.filterwarnings('ignore')

### Library Imports

In [1]:
%matplotlib widget

In [2]:
import numpy as np
from numba import njit
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import scipy.interpolate as interpolate
import scipy.integrate as integrate
from scipy.misc import derivative as derivative
from pynverse import inversefunc
import scipy.optimize as optimize
from uncertainties import ufloat
import itertools
import scipy

import ipywidgets as widgets
from IPython.display import display

# Personal libraries
import symplectic_map as sm
import henon_map as hm
import crank_nicolson_numba.nekhoroshev as nk
import crank_nicolson_numba.generic as cn

# animation
from matplotlib import animation, rc
from IPython.display import HTML

In [3]:
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

## Choose your parameters

In [14]:
# With epsilon = 16.0 we have approx DA(1000 turns) = 0.59 [a.u.]
epsilon = 16.0
samples = 1000
steps = 1000 # iterations to perform
bins = 100 # bins in the visualization system
r_0 = 0.50 # if we want to be at the DA we set 0.59 here

# Setup
I_0 = r_0 ** 2 / 2
r_sigma = 0.0001
r0 = np.random.normal(r_0, r_sigma, size=samples)

# We choose the angles as absolutely random!
alpha = np.random.rand(samples) * np.pi / 2
theta1 = np.random.rand(samples) * np.pi * 2
theta2 = np.random.rand(samples) * np.pi * 2
x = r0 * np.sin(alpha) * np.cos(theta1)
px = r0 * np.sin(alpha) * np.sin(theta1)
y = r0 * np.cos(alpha) * np.cos(theta2)
py = r0 * np.cos(alpha) * np.sin(theta2)

### Execute and interact with the result

In [13]:
henon_fig, henon_ax = plt.subplots()
print("Computing...")
r, a, th1, th2 = hm.cartesian_to_polar(x, px, y, py)
henon_engine = hm.full_track.generate_instance(
    r, a, th1, th2,
    np.ones(samples, dtype=np.int) * steps,
    epsilon
)
x, px, y, py = henon_engine.compute()

actions = (np.power(x, 2) + np.power(px, 2) + np.power(y, 2) + np.power(py, 2)) / 2

print("Done computing.")

def histo_test(index):
    henon_ax.clear()
    
    filtered_data = actions[index][np.logical_not(np.isnan(actions[index]))]
    weights = np.empty_like(filtered_data)
    ### USEFUL! ###
    weights.fill((filtered_data.size/actions[index].size) * bins / (filtered_data.max()-filtered_data.min()) / filtered_data.size)
    ###############
    n, _, _ = henon_ax.hist(filtered_data, weights=weights, bins=bins, range=(0, np.nanmax(actions)))
    
    #henon_ax.hist(actions[index, :], bins=bins, range=(0, np.max(actions)))
    henon_ax.set_xlim(0, np.nanmax(actions))
    henon_ax.set_ylim(0, 50)
    henon_ax.set_xlabel("$I$")
    henon_ax.set_ylabel("$\\rho(I)$") # We set this so that we have a stable view, but of course can be remodulated...
    henon_ax.set_title("Iterations $= {}$, $I_0={:.2}$, valid beam $={:.2}$".format(index, I_0, 1 - np.count_nonzero(np.isnan(actions[index])) / samples))
    
widj_idx_I = widgets.IntSlider(value=0, min=0, max=steps-1, step=1, continuous_update=True)

ui = widgets.VBox([
    widgets.Label("t sample"), widj_idx_I,
])
out = widgets.interactive_output(
    histo_test,
    {"index":widj_idx_I}
)

display(ui, out)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Computing...
Done computing.


Output()

**At a first glance, we do not observe a "proper" diffusion, but more an immediate relaxation into a distribution of more-or-less chaotic orbits that does not resembles immediatly a diffusive process.**